<a href="https://colab.research.google.com/github/NayabNoor01/Symptom-to-Disease-Diagnosis-and-Medicine-Recommendation-System/blob/main/Stage_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
dataset=files.upload()

Saving dataset.xlsx to dataset.xlsx


In [ ]:
import pandas as pd
dataset=pd.read_excel("dataset.xlsx")
print(dataset.head(5))

                                             Symptom  Disease  \
0  runny nose,frequent sneezing,itchy and watery ...  allergy   
1  sore throat,constant sneezing,swelling on the ...  allergy   
2  difficulty in breathing,constant coughing,itch...  allergy   
3  blocked nose,chest pain,chest tightness,loss o...  allergy   
4  difficulty in breathing,wheezing,fever,headach...  allergy   

                          Cause  \
0  pollen, food, or medications   
1  pollen, food, or medications   
2  pollen, food, or medications   
3  pollen, food, or medications   
4  pollen, food, or medications   

                                            Medicine  
0  ['Antihistamines (e.g., Loratadine)', 'Deconge...  
1  ['Antihistamines (e.g., Loratadine)', 'Deconge...  
2  ['Antihistamines (e.g., Loratadine)', 'Deconge...  
3  ['Antihistamines (e.g., Loratadine)', 'Deconge...  
4  ['Antihistamines (e.g., Loratadine)', 'Deconge...  


In [ ]:
from datasets import Dataset

#**Label Encoding**

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer


In [ ]:
multilabel = MultiLabelBinarizer()
cause_disease=dataset[['Cause','Disease']].values.tolist()
print(cause_disease)
labels=multilabel.fit_transform(cause_disease).astype('float32')
print(labels)

dataset['labels']=labels.tolist()
print(dataset.head(5))


[['pollen, food, or medications', 'allergy'], ['pollen, food, or medications', 'allergy'], ['pollen, food, or medications', 'allergy'], ['pollen, food, or medications', 'allergy'], ['pollen, food, or medications', 'allergy'], ['pollen, food, or medications', 'allergy'], ['pollen, food, or medications', 'allergy'], ['pollen, food, or medications', 'allergy'], ['pollen, food, or medications', 'allergy'], ['pollen, food, or medications', 'allergy'], ['pollen, food, or medications', 'allergy'], ['pollen, food, or medications', 'allergy'], ['pollen, food, or medications', 'allergy'], ['pollen, food, or medications', 'allergy'], ['pollen, food, or medications', 'allergy'], ['pollen, food, or medications', 'allergy'], ['pollen, food, or medications', 'allergy'], ['pollen, food, or medications', 'allergy'], ['pollen, food, or medications', 'allergy'], ['pollen, food, or medications', 'allergy'], ['pollen, food, or medications', 'allergy'], ['pollen, food, or medications', 'allergy'], ['pollen,

In [ ]:
dataset=Dataset.from_pandas(dataset[['Symptom','labels']])
split_dataset=dataset.train_test_split(test_size=0.2,seed=42)
train_dataset=split_dataset['train']
eval_dataset=split_dataset['test']


In [ ]:
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained('dmis-lab/biobert-base-cased-v1.1')

def tokenize_function(example):
  return tokenizer(example['Symptom'],padding='max_length',truncation=True)
train_dataset=train_dataset.map(tokenize_function,batched=True)
eval_dataset=eval_dataset.map(tokenize_function,batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
import numpy as np

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = 1 / (1 + np.exp(-logits))
    preds = (probs > 0.5).astype(int)

    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='samples')

    return {"accuracy": acc, "f1": f1}


In [ ]:
from transformers import AutoModelForSequenceClassification
num_labels = len(multilabel.classes_)
model=AutoModelForSequenceClassification.from_pretrained(
    'dmis-lab/biobert-base-cased-v1.1',
     problem_type="multi_label_classification",num_labels=num_labels
)

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments,Trainer
training_args=TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=400,
    weight_decay=0.01,
    eval_strategy='epoch',
    save_strategy='epoch',
    logging_dir='./logs',
    logging_steps=10,
    report_to='none',
)
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


/tmp/ipython-input-1001655134.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer=Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.169600,0.164190,0.700000,0.733333
2,0.162900,0.157907,0.730000,0.756667
3,0.154700,0.147888,0.750000,0.776667
4,0.139900,0.134784,0.800000,0.840000
5,0.128700,0.122775,0.930000,0.943333
6,0.112600,0.108700,0.970000,0.970000
7,0.102700,0.095753,0.980000,0.980000
8,0.086000,0.083788,0.980000,0.980000
9,0.077600,0.073565,0.980000,0.980000
10,0.065100,0.064292,0.980000,0.980000


TrainOutput(global_step=250, training_loss=0.12228453826904297, metrics={'train_runtime': 136.2726, 'train_samples_per_second': 29.353, 'train_steps_per_second': 1.835, 'total_flos': 76980645109632.0, 'train_loss': 0.12228453826904297, 'epoch': 10.0})

In [ ]:
trainer.save_model("./saved_model")
tokenizer.save_pretrained("./saved_model")

('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/vocab.txt',
 './saved_model/added_tokens.json',
 './saved_model/tokenizer.json')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Save to Drive instead of local Colab
trainer.save_model("/content/drive/MyDrive/saved_model")
tokenizer.save_pretrained("/content/drive/MyDrive/saved_model")


Mounted at /content/drive


('/content/drive/MyDrive/saved_model/tokenizer_config.json',
 '/content/drive/MyDrive/saved_model/special_tokens_map.json',
 '/content/drive/MyDrive/saved_model/vocab.txt',
 '/content/drive/MyDrive/saved_model/added_tokens.json',
 '/content/drive/MyDrive/saved_model/tokenizer.json')

In [ ]:
trainer.evaluate()

{'eval_loss': 0.06429167091846466,
 'eval_accuracy': 0.98,
 'eval_f1': 0.98,
 'eval_runtime': 0.2448,
 'eval_samples_per_second': 408.466,
 'eval_steps_per_second': 8.169,
 'epoch': 10.0}